# Lab 3 - Chemotaxis

This lab is designed to get you familiar with the basics of chemotaxis.  We will compare two of our random agents, levy and diffusion, with a gradient searcher who operates akin to a simple bacteria agent.

There are two goasl. 

1. Examine exploration for a single target with a variable scent in an open field. 
2. Play with a maze.


## Background

**Structured randomness and chemotaxis**

In lab 2 we played with a random search agent that wandered using Brownian motion. Here we will introuce a new random agent that moves according to Levy walks. 

* A Levy walk is is a continuing process of random movement where at each "step" of movement, a direction of and distance of movement is chosen randomly. The distance $\delta_i$ of movement at each time step $i$ is sampled from the random distribution as follows: $\delta_i = {u_i}^{-\frac{1}{\gamma}}$, where $u_i \sim N(\mu,\sigma)$ and $\gamma > 1$.

Our _chemotaxis_ agent (_AccumulatorGradientDiscrete_) tries to directly estimate the gradient of the scent $\nabla$  by comparing the level of scent at the last grid position it occupied to the current scent level ($o$). By last position here we mean the last position when it moved last. 

$$\nabla \approx o_t - o_{t-1}$$

Our chemotaxis agent thus behaves as follows:

- When the scent gradient is positive, meaning you are going "up" the gradient, the probability of turning is set to _p pos_. 
- When the gradient is negative, the turning probability is set to _p neg_. (See code below, for an example). 
- If the agent "decides" to turn, the direction is uniform random.
- The length of travel before the next turn decision is sampled from an exponential distribution just like the _DiffusionDiscrete_

## Section - Setup

Okay, let's now get things started by importing our libraries and functions.

In [ ]:
# # Install explorationlib
!pip install --upgrade git+https://github.com/parenthetical-e/explorationlib
!pip install --upgrade git+https://github.com/MattChanTK/gym-maze.git

In [ ]:
# Import misc
import shutil
import glob
import os
import copy
import sys

# Vis - 1
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Exp
from explorationlib.run import experiment
from explorationlib.util import select_exp
from explorationlib.util import load
from explorationlib.util import save

# Agents
from explorationlib.agent import DiffusionDiscrete
from explorationlib.agent import TruncatedLevyDiscrete
from explorationlib.agent import GradientDiffusionDiscrete

# Env
from explorationlib.local_gym import ScentGrid
from explorationlib.local_gym import ScentMazeEnv
from explorationlib.local_gym import create_grid_scent
from explorationlib.local_gym import create_maze_scent
from explorationlib.local_gym import create_grid_scent_patches


# Vis - 2
from explorationlib.plot import plot_position2d
from explorationlib.plot import plot_length_hist
from explorationlib.plot import plot_length
from explorationlib.plot import plot_targets2d
from explorationlib.plot import plot_scent_grid

# Score
from explorationlib.score import total_reward

In [ ]:
# Pretty plots
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.greedy=True
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = "16"

# Dev
%load_ext autoreload
%autoreload 2

## Section 2 - 

## Section 1 - singular scent
How much faster can smell get you there?

_Background_: the model of scent in our _sniff_ agent (aka _GradientDiffusionDiscrete_) is as simple as can be. 

- When the scent gradient is positive, meaning you are going "up" the gradient, the probability of turning is set to _p pos_. 
- When the gradient is negative, the turning probability is set to _p neg_. (See code below, for an example). 
- If the agent "decides" to turn, the direction is uniform random.
- The length of travel before the next turn decision is sampled from an exponential distribution just like the _DiffusionDiscrete_

_Note:_ this lab the open field and maze are defined on a discrete (integer) grid. In previous labs we worked with a continuous field. 



### Question 1.1
Make a blind guess for how much better the sniffing agent will be? Will the other random agents ever come close? Answer this for both the open field, with a single target, and the maze, with its walls barriers and dead ends.

In [ ]:
# Write your answer here, as a comment

The name of the env for this section is _ScentGrid_. Like adding targets, adding a scent is a separate step from creating the env. So the example code below.

### Example - run 1 experiment and visualize some of its results.

In [ ]:
# Experiment settings
num_experiments = 1
num_steps = 1000
p_neg = 1
p_pos = 0.5
scent_sigma = 10


# Env
detection_radius = 1
min_length = 1
max_length = 10

env = ScentGrid(mode="discrete")
boundary = (100, 100)
target = (5,5)
coord, scent = create_grid_scent(boundary, amplitude=1, sigma=scent_sigma)
env.add_scent(target, 1, coord, scent)
# TODO plot scent

# Agents
diff = DiffusionDiscrete(min_length=min_length, scale=1)
levy2 = TruncatedLevyDiscrete(min_length=min_length, max_length=max_length, exponent=2)
sniff = GradientDiffusionDiscrete(num_actions=4, min_length=min_length, scale=2, p_neg=p_neg, p_pos=p_pos)

# Cleanup 
for path in glob.glob("data/test4_*.pkl"):
    os.remove(path)

# !
levy2_exp = experiment(
    f"data/test4_levy.pkl",
    levy2,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
)
diff_exp = experiment(
    f"data/test4_diff.pkl",
    diff,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
)
sniff_exp = experiment(
    f"data/test4_sniff.pkl",
    sniff,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
)

Plot the scent 

_Note_: the axis is in matrix space not Grid space. Use this to get a sense of how high and wide the scent is. 

In [ ]:
plot_scent_grid(env)

Plot the walk (in grid space)

In [ ]:
plot_boundary = (100, 100)

num_experiment = 0
ax = plot_position2d(
    select_exp(levy2_exp, num_experiment),
    boundary=plot_boundary,
    label="Levy2",
    color="purple",
    alpha=0.6,
    figsize=(3, 3),
)
ax = plot_position2d(
    select_exp(diff_exp, num_experiment),
    boundary=plot_boundary,
    label="Diffusion",
    color="brown",
    alpha=0.6,
    ax=ax,
)
ax = plot_position2d(
    select_exp(sniff_exp, num_experiment),
    boundary=plot_boundary,
    label="Sniff",
    color="green",
    alpha=0.6,
    ax=ax,
)
ax = plot_targets2d(
    env,
    boundary=plot_boundary,
    color="black",
    alpha=1,
    label="Targets",
    ax=ax,
)   

Total reward

In [ ]:
print(f'Levy - {np.sum(select_exp(levy2_exp, num_experiment)["exp_reward"])}')
print(f'Diff - {np.sum(select_exp(diff_exp, num_experiment)["exp_reward"])}')
print(f'Sniff - {np.sum(select_exp(sniff_exp, num_experiment)["exp_reward"])}')

### Question 1.2
In the example above _p pos_ was 0.5, and _p neg_ was 1.0. This means that when the gradient was positive half the time the walker would change direction anyway. It also means the all the time when the grad was negative, the explorer would change direction.

Do these parameter choices seem optimal to you? 

Make a best guess for how to improve them, if you think they can be improved. Explain your choice.

In [ ]:
# Write your answer here, as a comment

### Question 1.3
Test your hypothesis from Question 1.2. Use total reward as your metric, and the code above to get started. 

_Note:_ Leave the _scent sigma_ parameter value set to 10.

Were you right? 

What is the best set of _p pos_ and _p neg_ that you can find?

In [ ]:
# Write your code here

In [ ]:
# Write your answers here, as a comment

### Question 1.4
In Question 1.3 we held _scent sigma_ set to 10. If we vary _scent sigma_ on (1,2,5,10), do you think this will change your best set of _p pos_ and _p neg_? 

Guess first, then test.

In [ ]:
# Write your answers here, as a comment

In [ ]:
# Write your code here

### Question 1.5
Was your hypothesis in Question 1.4 right?

What are the best sets of _p pos_ and _p neg_ you can find for the four _scent sigmas_ in Question 1.4?

In [ ]:
# Write your answers here, as a comment

## Section 2 - a-maze-zing 

I have modifed and existing [Maze env](https://github.com/MattChanTK/gym-maze) to have a scent. Here is an example of it, as a gif.

![Maze](https://camo.githubusercontent.com/0ddf75894c1afc047b54188c958c9eb6967b8b4d1148b95aae918113420a45ad/687474703a2f2f692e67697068792e636f6d2f417233614b786b4141683379302e676966)

The agent starts at the top (red), and tries to find the exit (blue) in the bottom right. In our version we can add a scent field to the exit. This can help our sniffer solve the maze more quickly, or at least that is what we guess should happen in principle.

### Example - one maze experiment

In [ ]:
# Experiment settings
num_experiments = 1
num_steps = 50000
p_neg = 1
p_pos = 0.5
scent_sigma = 5

# Env
detection_radius = 1
min_length = 1
max_length = 10

# Env
boundary = (10, 10)
env = ScentMazeEnv(maze_size=boundary)
coord, scent = create_maze_scent(boundary, amplitude=1, sigma=scent_sigma)
env.add_scent(scent)

# Agents
diff = DiffusionDiscrete(num_actions=4, min_length=min_length, scale=1)
levy2 = TruncatedLevyDiscrete(num_actions=4, min_length=min_length, max_length=max_length, exponent=2)
sniff = GradientDiffusionDiscrete(num_actions=4, min_length=min_length, scale=2, p_neg=p_neg, p_pos=p_pos)

# Cleanup 
for path in glob.glob("data/test4_*.pkl"):
    os.remove(path)

# !
levy2_exp = experiment(
    f"data/test4_levy.pkl",
    levy2,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
)
diff_exp = experiment(
    f"data/test4_diff.pkl",
    diff,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
)
sniff_exp = experiment(
    f"data/test4_sniff.pkl",
    sniff,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
)

Plot the experiment. 

_Note_: unfortunately out standard plotting methods “flip” the axis when displaying the maze. So it looks like we begin at the middle, and finish at the top right. Don't let this distract you. Nothing important has changed.

In [ ]:
plot_boundary = (10, 10)

num_experiment = 0
ax = plot_position2d(
    select_exp(levy2_exp, num_experiment),
    boundary=plot_boundary,
    label="Levy",
    color="purple",
    alpha=0.6,
    figsize=(3, 3),
)
ax = plot_position2d(
    select_exp(diff_exp, num_experiment),
    boundary=plot_boundary,
    label="Diff",
    color="brown",
    alpha=0.6,
    ax=ax,
)
ax = plot_position2d(
    select_exp(sniff_exp, num_experiment),
    boundary=plot_boundary,
    label="Sniff",
    color="green",
    alpha=0.6,
    ax=ax,
)

Total reward

_Note_: Postive values are good. Negative values bad. If you can't get positive values, try increasing _num steps_.

In [ ]:
print(f'Levy - {np.sum(select_exp(levy2_exp, num_experiment)["exp_reward"])}')
print(f'Diff - {np.sum(select_exp(diff_exp, num_experiment)["exp_reward"])}')
print(f'Sniff - {np.sum(select_exp(sniff_exp, num_experiment)["exp_reward"])}')

### Question 2.1

If we set _scent sigma_ to 5 for the maze, do you think the best _p pos_ and _p neg_ that you found in Question 1.5 will be best for the maze task? 

Why?

In [ ]:
# Write your answers here, as a comment

### Question 2.2

Do you think that *any* value of _p pos_ and _p neg_ will cause the sniffer to outperform the other two explorers (Levy and Diffusion)? Explain your answer.

In [ ]:
# Write your answers here, as a comment

### Question 2.3
To find an approximate answer to Question 2.2, run 100 experiments with three sets of _p pos_ and _p neg_ values. But first, explain your choice for each.

In [ ]:
# Write your answers here, as a comment

In [ ]:
# Write your code here

In [ ]:
# Analysis helper code:

# Results, names, and colors
results = [levy2_exp, diff_exp, sniff_exp]
names = ["Levy", "Diff", "Sniff"]
colors = ["purple", "brown", "green"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    r = total_reward(res)
    scores.append(r)   

In [ ]:
# Dists
for (name, s, c) in zip(names, scores, colors):
    plt.hist(s, label=name, color=c, alpha=0.5, bins=20)
    plt.legend()
    plt.xlabel("Score")
    plt.tight_layout()
    sns.despine()

In [ ]:
# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color="black", alpha=0.6)
plt.ylabel("Score")
plt.tight_layout()
sns.despine()